In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import config

In [ ]:
#Load google.cloud.bigquery
%load_ext google.cloud.bigquery

In [ ]:
#Select path to credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=config.GOOGLE_APPLICATION_CREDENTIALS

In [ ]:
%%bigquery --use_rest_api ZRI_MF
SELECT *
FROM `high-empire-220313.ZRI.Multi_Family`

In [ ]:
import pickle
file = open('pickles/ZRI_MF_wide.p','rb')
ZRI_MF = pickle.load(file)

In [ ]:
year_columns = [x for x in ZRI_MF.columns if ('20' in x)]

In [ ]:
ZRI_MF_long = ZRI_MF.melt(id_vars = ['RegionID','RegionName','City','State','Metro','CountyName','SizeRank'],
            value_vars = year_columns).rename({'value':'ZRI','variable':'Date'}, axis = 1)

In [ ]:
ZRI_MF_long.to_pickle('./pickles/ZRI_MF_long.p')

In [ ]:
ZRI_MF_long.loc[:,'Month'] = ZRI_MF_long['Date'].apply(lambda x: int(x[-2:]))
ZRI_MF_long.loc[:,'Year'] = ZRI_MF_long['Date'].apply(lambda x: int(x[1:5]))

In [ ]:
ZRI_MF_long.groupby('RegionName')[['ZRI']].mean()

In [ ]:
ZRI_std_mean = ZRI_MF_long.groupby('RegionName')['ZRI'].agg({'mean', 'std'})

In [ ]:
%%bigquery --use_rest_api ZIPS
SELECT *
FROM `high-empire-220313.InfoByZipcode.USzips`

In [ ]:
%%bigquery --use_rest_api ZIP_Area
SELECT *
FROM `high-empire-220313.InfoByZipcode.Zipcode_Water_Land_area_lat_long_county_zipcodegeom`

In [ ]:
ZIP_Area.loc[:,'zipcode'] = ZIP_Area.zipcode.astype(int)

In [ ]:
ZRI_ZIP_Area = ZRI_MF_long.merge(ZIPS,how = 'left', left_on = 'RegionName',right_on = 'zip')

In [ ]:
ZRI_ZIP_Area= ZRI_ZIP_Area.merge(ZIP_Area[['area_land_meters','area_water_meters','area_land_miles','area_water_miles','zipcode']],
         how = 'left',
         left_on = 'zip',
         right_on = 'zipcode')